In [1]:
using VarLMM, Random, LinearAlgebra, Distributions, DataFrames, CSV
@info "generate data"
Random.seed!(123)
# dimensions
m  = 500 # number of individuals
ns = rand(9:11, m) # numbers of observations per individual
p  = 5    # number of fixed effects, including intercept
q  = 2    # number of random effects, including intercept
l  = 4    # number of WS variance covariates, including intercept
obsvec = Vector{VarLmmObs{Float64}}(undef, m)
# true parameter values
βtrue = [100.0; 15.0; 10.0; 0.25; -10.0] #intercept, agegroup, #gender, #bmi, #meds
τtrue = [-2.5; 1.5; -0.5; 0.0] #intercept, agegroup, meds, bmi
Σγ    = Matrix(Diagonal([1.0; 0.0]))
δγω   = [0.0; 0.0; zeros(q - 2) ./ 10]
σω    = [0.0]
Σγω   = [Σγ δγω; δγω' σω]
Lγω   = cholesky(Symmetric(Σγω), check = false).L
Lγ    = Lγω[1:q, 1:q]
lγω   = Lγω[q + 1, 1:q]
lω    = Lγω[q + 1, q + 1]
# generate data
γω = Vector{Float64}(undef, q + 1)
z  = similar(γω) # hold vector of iid std normal
df = DataFrame(id = String[], sbp = Float64[], agegroup = Float64[], 
    gender=Int64[], bmi = Float64[], meds = Float64[])
for i in 1:m
    # first column intercept, remaining entries iid std normal
    X = Matrix{Float64}(undef, ns[i], p)
    X[:, 1] .= 1
    agegroup = Distributions.rand(1:3) #age
    gender = Distributions.rand(Bernoulli(0.5)) #gender
    meds = Distributions.rand(Bernoulli(0.2)) #meds
    bmi = Distributions.rand(Normal(25, 1.2), ns[i])
    @views fill!(X[:, 2], agegroup)
    @views fill!(X[:, 3], Int(gender))
    @views copyto!(X[:, 4], bmi)
    @views fill!(X[:, 5], meds)
    # first column intercept, remaining entries iid std normal
    Z = Matrix{Float64}(undef, ns[i], q)
    Z[:, 1] .= 1
    @views copyto!(Z[:, 2], X[:, 4]) #bmi 
    # first column intercept, remaining entries iid std normal
    W = Matrix{Float64}(undef, ns[i], l)
    W[:, 1] .= 1
    @views copyto!(W[:, 2], X[:, 2]) #agegroup
    @views copyto!(W[:, 3], X[:, 5]) #meds
    @views copyto!(W[:, 4], X[:, 4]) #bmi
    # generate random effects: γω = Lγω * z
    mul!(γω, Lγω, Distributions.rand!(Normal(), z))
    # generate y
    μy = X * βtrue + Z * γω[1:q]
    @views vy = exp.(W * τtrue .+ dot(γω[1:q], lγω) .+ γω[end])
    y = rand(MvNormal(μy, Diagonal(vy)))
        if i == 8
        @show vy
        @show μy
        @show y
    end
    id = fill(string(i), ns[i])
    tempdf = DataFrame([id y X[:, 2:p]])
    rename!(tempdf, [:id, :sbp, :agegroup, :gender, :bmi, :meds])
    # form a VarLmmObs instance
    append!(df, tempdf)
end
rename!(df, [:id, :sbp, :agegroup, :gender, :bmi, :meds])

┌ Info: generate data
└ @ Main In[1]:2


vy = [7.38905609893065, 7.38905609893065, 7.38905609893065, 7.38905609893065, 7.38905609893065, 7.38905609893065, 7.38905609893065, 7.38905609893065, 7.38905609893065]
μy = [160.73568614876524, 161.12726738169906, 160.7523165946845, 160.7897198908203, 160.3457789407866, 160.27668438054704, 160.90340388121535, 160.5821039860477, 161.0410901880801]
y = [155.92040542556265, 164.06032803785934, 160.3399467983824, 157.3128981314478, 163.2894577026046, 159.60558405082082, 159.88272843704567, 165.7592300539918, 163.30987783782464]


,id,sbp,agegroup,gender,bmi,meds
,String,Float64,Float64,Int64,Float64,Float64
1,1,159.586,3.0,1,23.1336,0.0
2,1,161.849,3.0,1,26.5885,0.0
3,1,160.484,3.0,1,24.8428,0.0
4,1,161.134,3.0,1,24.9289,0.0
5,1,165.443,3.0,1,24.8057,0.0
6,1,160.053,3.0,1,24.1583,0.0
7,1,162.1,3.0,1,25.2543,0.0
8,1,163.153,3.0,1,24.3951,0.0
9,1,166.675,3.0,1,26.1514,0.0


In [2]:
describe(df)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,id,,1,,99,500,,String
2,sbp,140.365,109.865,139.44,171.545,,,Float64
3,agegroup,2.05388,1.0,2.0,3.0,,,Float64
4,gender,0.492716,0,0.0,1,,,Int64
5,bmi,25.028,20.7168,25.0226,29.6457,,,Float64
6,meds,0.166633,0.0,0.0,1.0,,,Float64


In [3]:
df[!, :gender] = map(x -> x == 1 ? "Male" : "Female", df[!, :gender]);
df[!, :meds] = map(x -> x == 1 ? "OnMeds" : "NoMeds", df[!, :meds]);

In [4]:
describe(df)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,id,,1,,99,500,,String
2,sbp,140.365,109.865,139.44,171.545,,,Float64
3,agegroup,2.05388,1.0,2.0,3.0,,,Float64
4,gender,,Female,,Male,2,,String
5,bmi,25.028,20.7168,25.0226,29.6457,,,Float64
6,meds,,NoMeds,,OnMeds,2,,String


In [5]:
vlmm = VarLmmModel(@formula(sbp ~ 1 + agegroup + gender + bmi + meds), 
    @formula(sbp ~ 1 + bmi), 
    @formula(sbp ~ 1 + agegroup + meds + bmi),
    :id, df);

In [6]:
fit!(vlmm)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       28

Total number of variables............................:        7
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equ

───────────────────────────────────────────────────────────────────
                  Estimate  Std. Error  Wald Statistic  Pr(>|Wald|)
───────────────────────────────────────────────────────────────────
(Intercept)   100.077        0.346108         83606.99       <1e-99
agegroup       14.9864       0.063344         55973.45       <1e-99
gender: Male   10.0725       0.100267         10091.56       <1e-99
bmi             0.248811     0.0126703          385.62       <1e-85
meds: OnMeds  -10.115        0.123701          6686.30       <1e-99
(Intercept)    -2.63253      1.0034               6.88       0.0087
agegroup        1.50585      0.0567203          704.84       <1e-99
meds: OnMeds   -0.426159     0.0535525           63.33       <1e-14
bmi             0.00434957   0.0472932            0.01       0.9267
───────────────────────────────────────────────────────────────────

In [7]:
CSV.write("varlmm_exdata.csv", df)

"varlmm_exdata.csv"